In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df=  pd.read_csv('..\data\Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
def df_shape(temp):
    print(temp.shape)

In [4]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
df= df.drop(columns=['RowNumber','CustomerId','Surname'])


In [6]:
# df_2= pd.get_dummies(df,columns=['Geography'],drop_first=True)
df_2= df.copy()
# df_2.head(2)
df.shape

(10000, 11)

In [7]:
enc = OneHotEncoder(drop='first')


In [8]:
val = enc.fit_transform(df[['Geography']]).toarray()
val_gender = enc.fit_transform(df[['Gender']]).toarray()
# df[['mew_f','new_s','new_g']]= val

In [9]:
df[['new_s','new_g']]=val
df[['male']]=val_gender

In [10]:
df.drop(columns=['Geography','Gender'],axis=0,inplace=True)

In [11]:
df.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,new_s,new_g,male
0,619,42,2,0.00,1,1,1,101348.88,1,0.0,0.0,0.0
1,608,41,1,83807.86,1,0,1,112542.58,0,0.0,1.0,0.0
2,502,42,8,159660.80,3,1,0,113931.57,1,0.0,0.0,0.0
3,699,39,1,0.00,2,0,0,93826.63,0,0.0,0.0,0.0
4,850,43,2,125510.82,1,1,1,79084.10,0,0.0,1.0,0.0


In [12]:
# sns.scatterplot(df,y='Exited',x='NumOfProducts',hue= 'Tenure')
# plt.pie(df['Exited'])

In [13]:
le= MinMaxScaler()

In [14]:
df['uBalance']= le.fit_transform(df[['Balance']])

In [15]:
df['uEstimatedSalary']= le.fit_transform(df[['EstimatedSalary']])

In [16]:
# tenure= df[df['Tenure']!=0]
# tenure['Balance'].min()
val = df[ df['Age']> 0]
val = val[val['Balance']!=0]
val.shape
df_2= val.copy()
x= df.drop(columns=['Exited','Balance','EstimatedSalary'])
y= df[['Exited']]
y= np.array(y).reshape(-1,1)


In [17]:
# sns.scatterplot(val,x='Balance',y='Age',hue='Exited')

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)


In [19]:
model_1 = keras.Sequential([
    Dense(16,activation='relu'),
    Dense(16,activation='relu'),
    Dense(8,activation='relu'),
    Dense(1,activation='sigmoid')
])

In [20]:
# model_1.evaluate(x_test,y_test)

In [21]:
# batch = 75, epoch= 30, iteration= 7500/75

In [22]:
lr=keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=10000,
    decay_rate=0.96)


In [23]:
model_1.compile(optimizer=tf.optimizers.Adam(learning_rate=lr,),
                loss= tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [24]:
x_train.shape

(7500, 11)

In [25]:
model_1.fit(x_train,y_train,batch_size=75,epochs= 30,validation_data=(x_test,y_test)  )

Epoch 1/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7325 - loss: 1.2651 - val_accuracy: 0.7992 - val_loss: 0.6222
Epoch 2/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7748 - loss: 0.6362 - val_accuracy: 0.8012 - val_loss: 0.5649
Epoch 3/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7889 - loss: 0.5819 - val_accuracy: 0.8012 - val_loss: 0.5818
Epoch 4/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7966 - loss: 0.5310 - val_accuracy: 0.8012 - val_loss: 0.4949
Epoch 5/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7961 - loss: 0.5185 - val_accuracy: 0.8012 - val_loss: 0.4967
Epoch 6/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7966 - loss: 0.5139 - val_accuracy: 0.8012 - val_loss: 0.4736
Epoch 7/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7946 - loss: 0.4842 - val_accuracy: 0.7920 - val_loss: 0.5394
Epoch 8/30
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7936 - loss: 0.4889 - val_accuracy: 0.